In [1]:
%matplotlib inline
#%matplotlib notebook
import numpy as np
from matplotlib import pyplot as plt

import time
import csv
import h5py
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import BatchNormalization, Reshape, UpSampling2D, Conv2DTranspose, LeakyReLU, ZeroPadding2D, Input, MaxPooling2D, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.recurrent import LSTM, GRU
from keras.models import Sequential, load_model, Model
from keras import regularizers, optimizers, initializers, constraints, metrics, objectives
from keras.optimizers import RMSprop, Adam
from keras.engine import InputSpec, Layer, InputLayer
from keras.regularizers import l2
np.random.seed(1234)
from keras.utils import plot_model
import keras.backend.tensorflow_backend as Kback
from IPython.display import clear_output
import keras as K
import tensorflow as tf
import scipy.io as sio
from scipy import stats
import scipy as scp
from scipy.signal import resample
import keras
from keras.backend.tensorflow_backend import set_session
import os
import random
import time
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='1'
config=tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction=1
set_session(tf.Session(config=config))

def set_trainable(model, trainable):
    model.trainable = trainable
    for layer in model.layers:
        layer.trainable = trainable

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/sam/anaconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/sam/anaconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/sam/anaconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/sam/anaconda3/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/home/sam/anaconda3/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.23' not found (required by /home/sam/anaconda3/lib/python3.7/site-packages/tensorf

In [ ]:
#Load data
#f = h5py.File('cropped_celeb.mat')
#size= 65534
f = h5py.File('cropped.mat')
size = 13775

In [ ]:
#input dimensions
hrz = 64
vrt = 64
#initialize arrays
x_train = {}
x_train_small_pre = np.ndarray((size,hrz,vrt,3),int)
x_train_small = np.ndarray((size,hrz,vrt,3),int)
x_train = f['bigdata'][()]
print(x_train.shape)
x_train_s = np.swapaxes(x_train,0,3)
x_train_s = np.swapaxes(x_train_s,1,2)
print(x_train_s.shape)

In [ ]:
for j in range(0,x_train_s.shape[0]):
    pic = scp.misc.imresize(x_train_s[j,:,:,:],(hrz,vrt,3))
    x_train_small_pre[j,:,:,:] = pic 
#x_train_small_pre = x_train_s
print(pic[1,1,1])

In [ ]:
print(x_train_small_pre.shape)
x_train_small_pre = np.random.permutation(x_train_small_pre)
print(x_train_small_pre.shape)

In [ ]:
probe = 1
testpic = x_train_small_pre[probe,:,:,:]
print(testpic.shape)
plt.imshow(testpic.astype(np.uint8))
print(np.amax(testpic))
print(np.amin(testpic))

In [ ]:
#normalize images between -1 and 1
x_train_small = (x_train_small_pre-127.5)/127.5

testpic_sml = x_train_small[probe,:,:,:]
print("max:",np.amax(testpic_sml[:,:,2]))
print("min:",np.amin(testpic_sml[:,:,2]))

rdiff = np.ones([hrz*vrt])
gdiff = np.ones([hrz*vrt])
bdiff = np.ones([hrz*vrt])
rabsdiff= np.ones([4])
gabsdiff= np.ones([4])
babsdiff= np.ones([4])

#get an idea of raw pixel similarities to later check for mode collapse
for sample_in in range(4):
    intgr = 0
    for xax in range(hrz):
        for yax in range(vrt):
            rdiff[intgr] = x_train_small[0,xax,yax,0]-x_train_small[sample_in,xax,yax,0]
            gdiff[intgr] = x_train_small[0,xax,yax,1]-x_train_small[sample_in,xax,yax,1]
            bdiff[intgr] = x_train_small[0,xax,yax,2]-x_train_small[sample_in,xax,yax,2]
            intgr = intgr+1
    rabsdiff[sample_in] = 1-np.mean(abs(rdiff))
    gabsdiff[sample_in] = 1-np.mean(abs(gdiff))
    babsdiff[sample_in] = 1-np.mean(abs(bdiff))
      
print("similarity:",np.mean([rabsdiff,gabsdiff,babsdiff]))

testpic = ((x_train_small[probe,:,:,:]*127.5)+127.5)
#testpic = x_train_small[probe,:,:,:]*255
print("max:",np.amax(testpic[:,:,2]))
print("min:",np.amin(testpic[:,:,2]))
#check if we can reproduce image after normalization
plt.imshow(testpic.astype(np.uint8)) 

In [ ]:
class DCGAN():
    def __init__(self,param):
        self.img_rows =hrz #image height
        self.img_cols = vrt #image width
        self.channels = 3 #RGB channels
        self.img_shape = (self.img_rows,self.img_cols,self.channels)
        self.latent_dim = 512 #dimension of latent vector
        self.wdecay = 1e-5 #weight decay for l2 normalization (smaller weights generate simpler model)
        self.recon_vs_gan_weight=1 #1e-4
        learning_rate= 0.0005
        rmsprop = RMSprop(lr = learning_rate)
        #rmsprop = Adam(lr = learning_rate, beta_1 = 0.5, beta_2 = 0.999)
        
        self.encoder = self.build_encoder(int(param[0]))        
        self.decoder = self.build_generator(int(param[0]))
        self.discriminator = self.build_discriminator(int(param[0]))
        
        #Build graphs
        #Define input shapes
        X_im = Input(shape=(self.img_shape))
        noise = Input(shape=(self.latent_dim,))
        
        #encoder takes a picture and spits out a latent vector
        z_mean, z_log_var, z = self.encoder(X_im)
        
        #the decoder either takes a random latent vector or a latent vector generated by the encoder
        x_gen = self.decoder(z) #encoder latent vector
        x_p = self.decoder(noise) #random latent vector

        dis_x, dis_feat_x = self.discriminator(X_im) #deciding on real images
        dis_x_gen, dis_feat_gen = self.discriminator(x_gen) #deciding on decoded real images
        dis_x_p  = self.discriminator(x_p)[0] #deciding on decoded fake images
        
        #how to punish the encoder
        #Does the activation in the Nth layer of the discriminator look the same for a real picture
        #and a picture that you encoded (and the generator decoded)?
        #bincross_loss = self.enc_binary_crossentropy(dis_feat_x, dis_feat_gen)
        layer_loss = self.MSE(dis_feat_x, dis_feat_gen)*5
        
        #enc_recon_loss = self.mean_gaussian_negative_log_likelihood(dis_feat_x, dis_feat_gen)
        
        #do the generated probability distributions of the latent space differ a lot from the normal distribution?
        #this forces the encoder to make highly similar == compact representations
        KL_loss = self.kl_loss(z_mean,z_log_var)
        
        #takes an image and spits out the activation of the discriminator at layer N (in response to any picture)
        self.encoder_train = Model([X_im],[dis_feat_gen]) 
        self.encoder_train.add_loss(KL_loss)
        self.encoder_train.add_loss(layer_loss)       
       
        #takes in an image or a random latent vector and spits out a real decoded or a fake generated image
        #this guy also gets punished if the discriminator activates differently for real versus fake decoded images
        
        self.decoder_train = Model([X_im,noise], [dis_x_gen, dis_x_p])
        #optional
        #self.decoder_train.add_loss(layer_loss)
                
        #this guy takes either a real image or a (noise vector??) and spits out a decision 
        #and activations of the Nth layer for decoded images or real images.
        #we can trace the variable [noise] back.
        self.discriminator_train = Model([X_im, noise], [dis_x, dis_x_gen, dis_x_p])
            
        set_trainable(self.encoder, False)
        set_trainable(self.decoder, False)
        self.discriminator_train.compile(rmsprop, ['binary_crossentropy']*3)
                          
        set_trainable(self.discriminator, False)
        set_trainable(self.decoder, True)
        self.decoder_train.compile(rmsprop, ['binary_crossentropy']*2)
        
        #self.decoder_train.metrics_tensors.append(layer_loss)
        #self.decoder_train.metrics_names.append("layer_loss")
                          
        set_trainable(self.decoder, False)
        set_trainable(self.encoder, True)
        self.encoder_train.compile(rmsprop, ['binary_crossentropy'], loss_weights = [0])
        
        self.encoder_train.metrics_tensors.append(KL_loss)
        self.encoder_train.metrics_names.append("KL_loss")

        self.encoder_train.metrics_tensors.append(layer_loss)
        self.encoder_train.metrics_names.append("layer_loss")
        #self.encoder_train.compile(optimizer = rmsprop)

                          
    #Build Encoder
    def build_encoder(self,depth):
        dropout = 0 #0.5
        input_shape = (hrz,vrt,3)
        disax = -1
        
        X = Input(shape=input_shape)
        #Just a bunch of convolutional layers.
        #Batch normalization makes sure that each layers output has more or less the same distribution 
        #to not confuse the next layer over the course of the training.
        model = Conv2D(depth*1,kernel_size =  5, strides=2,input_shape = input_shape, padding ='same', kernel_regularizer=l2(self.wdecay))(X)
        model = BatchNormalization(axis = disax, momentum=0.8)(model)
        model = LeakyReLU(alpha = 0.2)(model)
        model = Dropout(dropout)(model)

        model = Conv2D(depth*2,kernel_size =  5, strides=2, padding ='same', kernel_regularizer=l2(self.wdecay))(model)
        model = BatchNormalization(axis = disax, momentum=0.8)(model)
        model = LeakyReLU(alpha = 0.2)(model)
        model = Dropout(dropout)(model)

        model = Conv2D(depth*4,kernel_size =  5, strides=2, padding ='same', kernel_regularizer=l2(self.wdecay))(model)
        model = BatchNormalization(axis = disax, momentum=0.8)(model)
        model = LeakyReLU(alpha = 0.2)(model)
        model = Dropout(dropout)(model)
        
        model = Conv2D(depth*4,kernel_size =  5, strides=2, padding ='same', kernel_regularizer=l2(self.wdecay))(model)
        model = BatchNormalization(axis = disax, momentum=0.8)(model)
        model = LeakyReLU(alpha = 0.2)(model)
        model = Dropout(dropout)(model)
        
        model = Conv2D(depth*8,kernel_size =  5, strides=2, padding ='same', kernel_regularizer=l2(self.wdecay))(model)
        model = BatchNormalization(axis = disax, momentum=0.8)(model)
        model = LeakyReLU(alpha = 0.2)(model)
        model = Dropout(dropout)(model)
        
#         model = Conv2D(depth*8,kernel_size =  5, strides=2, padding ='same',kernel_regularizer=l2(self.wdecay))(model)
#         model = BatchNormalization(axis = disax, momentum=0.8)(model)
#         model = LeakyReLU(alpha = 0.2)(model)
#         model = Dropout(dropout)(model)
        
        model = Flatten()(model) 
        
        #This is the variational part of the autoencoder
        #Instead of just compressing into a meaningful Z we will learn the mean and the std of every value in Z
        #This teaches the network that features in Z are probabilistic
        z_mean = Dense(self.latent_dim,name ='z_mean')(model)
        z_log_var = Dense(self.latent_dim,name ='z_var')(model)
        #here we sample from the distributions in Z
        z= Lambda(self.sampling, output_shape=(self.latent_dim,))([z_mean, z_log_var])
        #the encoding model receives images and spits out Z
        encoder_model = Model([X],[z_mean,z_log_var,z])
        return encoder_model
        
    #Build generator
    def build_generator(self,depth):
        dropout  = 0 #0.5
        #some stacked conv2dtranspose layers
        generator_model = Sequential(name='generator')
        
        generator_model.add(InputLayer(input_shape=(self.latent_dim,)))
        generator_model.add(Reshape((1,1,self.latent_dim)))
        
        generator_model.add(Conv2DTranspose(depth*8,kernel_size = 5, strides = 2, padding ='same', kernel_regularizer=l2(self.wdecay)))
        generator_model.add(Dropout(dropout))
        generator_model.add(BatchNormalization(momentum=0.8))
        generator_model.add(LeakyReLU(alpha = 0.2))
        
#         generator_model.add(Conv2DTranspose(depth*8,kernel_size = 5, strides = 2, padding ='same', kernel_regularizer=l2(self.wdecay)))
#         generator_model.add(Dropout(dropout))
#         generator_model.add(BatchNormalization(momentum=0.8))
#         generator_model.add(LeakyReLU(alpha = 0.2))
        
        generator_model.add(Conv2DTranspose(depth*4,kernel_size = 5, strides = 2, padding ='same', kernel_regularizer=l2(self.wdecay)))
        generator_model.add(Dropout(dropout))
        generator_model.add(BatchNormalization(momentum=0.8))
        generator_model.add(LeakyReLU(alpha = 0.2))
        
        generator_model.add(Conv2DTranspose(depth*4,kernel_size = 5, strides = 2, padding ='same', kernel_regularizer=l2(self.wdecay)))
        generator_model.add(Dropout(dropout))
        generator_model.add(BatchNormalization(momentum=0.8))
        generator_model.add(LeakyReLU(alpha = 0.2))
        
        generator_model.add(Conv2DTranspose(depth*2,kernel_size = 5, strides = 2, padding ='same', kernel_regularizer=l2(self.wdecay)))
        generator_model.add(Dropout(dropout))
        generator_model.add(BatchNormalization(momentum=0.8))
        generator_model.add(LeakyReLU(alpha = 0.2))
        
        generator_model.add(Conv2DTranspose(depth*1,kernel_size = 5, strides = 2, padding ='same', kernel_regularizer=l2(self.wdecay)))
        generator_model.add(Dropout(dropout))
        generator_model.add(BatchNormalization(momentum=0.8))
        generator_model.add(LeakyReLU(alpha = 0.2))
        
        generator_model.add(Conv2DTranspose(self.channels,kernel_size = 5, strides = 2, padding ='same', kernel_regularizer=l2(self.wdecay)))
        generator_model.add(Activation("tanh"))
        #the generator receives a latent vector Z and spits out a 128,128,3 image
        return generator_model
    
    #Build Discriminator
    def build_discriminator(self,depth):
        dropout = 0 #0.5
        input_shape = (hrz,vrt,3)
        #almost identical architecture as the encoder
        X = Input(shape=input_shape)

        modelout = Conv2D(depth*1,kernel_size =  5, strides=2,input_shape = input_shape, padding ='same', kernel_regularizer=l2(self.wdecay))(X)
        model = BatchNormalization(momentum=0.8)(modelout)
        model = LeakyReLU(alpha = 0.2)(model)
        model = Dropout(dropout)(model)
        
        model = Conv2D(depth*2,kernel_size =  5, strides=2, padding ='same', kernel_regularizer=l2(self.wdecay))(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha = 0.2)(model)
        model = Dropout(dropout)(model)

        model = Conv2D(depth*4,kernel_size =  5, strides=2, padding ='same', kernel_regularizer=l2(self.wdecay))(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha = 0.2)(model)
        model = Dropout(dropout)(model)
        
        #here we save the output of the layer as separate variable
        model = Conv2D(depth*4,kernel_size =  5, strides=2, padding ='same', kernel_regularizer=l2(self.wdecay))(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha = 0.2)(model)
        model = Dropout(dropout)(model)

        model = Conv2D(depth*8,kernel_size =  5, strides=2, padding ='same', kernel_regularizer=l2(self.wdecay))(model)
        
#         model = BatchNormalization(momentum=0.8)(model)
#         model = LeakyReLU(alpha = 0.2)(model)
#         model = Dropout(dropout)(model)
#         model = Conv2D(depth*8,kernel_size =  5, strides=2, padding ='same',kernel_regularizer=l2(self.wdecay))(model)
        
        dec = BatchNormalization(momentum=0.8)(model)
        dec = LeakyReLU(alpha = 0.2)(dec)
        dec = Dropout(dropout)(dec)
        
        dec = Flatten()(dec)
        dec = Dense(1,activation='sigmoid', kernel_regularizer=l2(self.wdecay))(dec)
        #discriminator takes an image as input and spits out a decision (0 or 1) and its own activation at layer N
        output = Model([X], [dec, modelout])
        
        return output
    
    def sampling(self,args):
        z_mean,z_log_var = args
        epsilon = Kback.random_normal(shape = (Kback.shape(z_mean)[0],self.latent_dim), mean=0, stddev=1) 
        return z_mean + Kback.exp(z_log_var/2)*epsilon
    
    def enc_binary_crossentropy(self,y_true, y_pred):
        norm = 5
        y_true = Flatten()(y_true)
        y_pred = Flatten()(y_pred)
        return Kback.mean(Kback.mean(Kback.binary_crossentropy(y_true, y_pred), axis=-1))*norm
    
    def MSE(self,y_true, y_pred):
        y_true = Flatten()(y_true)
        y_pred = Flatten()(y_pred)
        return Kback.mean(Kback.mean(Kback.square(y_pred - y_true), axis=-1))
    
    def kl_loss(self,z_log_var,z_mean):
        kl_ls = Kback.mean(Kback.mean(-0.5 * Kback.sum(1 + z_log_var - Kback.square(z_mean) - Kback.exp(z_log_var), axis=-1)))
        return kl_ls
    
    def mean_gaussian_negative_log_likelihood(self,y_true, y_pred):
        y_true = Flatten()(y_true)
        y_pred = Flatten()(y_pred)
        nll = 0.5 * np.log(2 * np.pi) + 0.5 * Kback.square(y_pred - y_true)
        axis = tuple(range(1, len(Kback.int_shape(y_true))))
        return Kback.mean(Kback.sum(nll, axis=axis), axis=-1)

In [ ]:
#tRAINING
epochs =3000
batch_size = 100

loopits = 1
param1 = np.ones([2,loopits])
fake_samples =  np.ones([loopits,4,hrz,vrt,3])
fake_samples_training = np.ones([int(epochs/500),9,hrz,vrt,3])
scl = 32
plotstuff = 1
param1[0][:] = [scl*1] 
param1[1][:] = [scl*1] 

samp_ind = -1
real_index = -1
fake_index = -1

dis_loss = []
dec_loss = []
enc_loss = []

similarity = np.ones([int(epochs/500)])
idx = 0
drop = -1
for looper in range(loopits):
    DCGAN_model = DCGAN(param1[:,looper])
    DCGAN_model.encoder.summary()
    DCGAN_model.decoder.summary()
    DCGAN_model.discriminator.summary()
    #plot_model(DCGAN_model.encoder, to_file='encoder.png')
    #plot_model(DCGAN_model.decoder, to_file='decoder.png')
    #plot_model(DCGAN_model.discriminator, to_file='discriminator.png')
    y_true = np.ones([batch_size,1])
    y_false = np.zeros([batch_size,1])
    
    for epoch in range(epochs):
        clear_output(wait=True)
        #print(looper)
        print(epoch)
        
        if idx+batch_size>x_train_small.shape[0]:
            idx = 0
        imgs = x_train_small[range(idx,idx+batch_size)]
        idx = idx+batch_size
        
        noise = np.random.normal(0,1.0, (int(batch_size),DCGAN_model.latent_dim))
        
        #dis_x = discriminator_binary decision on real
        #dis_x_gen = discriminator bonary decision on fake
        #dis_feat_p = discriminator last layer output on real
        #dis_feat_gen = discriminator last layer output on fake
                                                                                #real,decoded real,decoded fake
        dis_Loss = DCGAN_model.discriminator_train.train_on_batch([imgs,noise],[y_true, y_false, y_false])
        dec_Loss = DCGAN_model.decoder_train.train_on_batch([imgs,noise],[y_true, y_true])
        [dis_x, dis_feat_gen] = DCGAN_model.discriminator.predict(imgs)
        enc_Loss = DCGAN_model.encoder_train.train_on_batch([imgs],[dis_feat_gen])

        dis_loss.append(dis_Loss)
        dec_loss.append(dec_Loss)
        enc_loss.append(enc_Loss)
        
#         dis_loss[looper,epoch,:] = DCGAN_model.discriminator_train.train_on_batch([imgs,noise],[y_true, y_false, y_false])
#         dec_loss[looper,epoch,:] = DCGAN_model.decoder_train.train_on_batch([imgs,noise],[y_true, y_true])
#         [dis_x, dis_feat_gen] = DCGAN_model.discriminator.predict(imgs)
#         enc_loss[looper,epoch,:] = np.mean(DCGAN_model.encoder_train.train_on_batch([imgs],[dis_feat_gen]))
        
        #discriminator: binary crossentropy                3 value prediction disc(real,decoded,fake) x100
        #               metrics                            3
        
        #decoder:       binary crossentropy norm           2 value prediction disc(real,fake)         x100
        #               metrics                            2
        
        #encoder:       binary crossentropy                1 value mean(4x4x128) disc activations     x100
        #               kl_div                             1 value                                    x100
        if epoch%500==0:
            drop = drop+1
            for sample in range(9):
                noise = np.random.normal(0,1.0, size=[batch_size,DCGAN_model.latent_dim])
                images_good_fake = DCGAN_model.decoder.predict(noise)
                fake_sample = (np.squeeze(images_good_fake[1,:,:,:]))  
                fake_samples_training[drop,sample,:,:,:] = fake_sample
fake_samples_training = (fake_samples_training+1)/2

In [ ]:
width = 10
height = 10
plt.figure(figsize=(width,height))
fake_samples =  np.ones([loopits,9,hrz,vrt,3])
looper = 0;
for sample in range(9):
    noise = np.random.normal(0,1.0, size=[batch_size,DCGAN_model.latent_dim])
    images_good_fake = DCGAN_model.decoder.predict(noise)
    fake_sample = (np.squeeze(images_good_fake[1,:,:,:]))  
    fake_samples[looper,sample,:,:,:] = fake_sample
    
fake_samples_scaled = (fake_samples+1)/2
plotcounter = 1    
for looper in range(loopits):
 
    plt.subplot(3,3,plotcounter)
    plt.imshow(fake_samples_scaled[looper,0,:,:])
    plt.subplot(3,3,plotcounter+1)
    plt.imshow(fake_samples_scaled[looper,1,:,:])
    plt.subplot(3,3,plotcounter+2)
    plt.imshow(fake_samples_scaled[looper,2,:,:])
    plt.subplot(3,3,plotcounter+3)
    plt.imshow(fake_samples_scaled[looper,3,:,:])
    plt.subplot(3,3,plotcounter+4)
    plt.imshow(fake_samples_scaled[looper,4,:,:])
    plt.subplot(3,3,plotcounter+5)
    plt.imshow(fake_samples_scaled[looper,5,:,:])
    plt.subplot(3,3,plotcounter+6)
    plt.imshow(fake_samples_scaled[looper,6,:,:])
    plt.subplot(3,3,plotcounter+7)
    plt.imshow(fake_samples_scaled[looper,7,:,:])
    plt.subplot(3,3,plotcounter+8)
    plt.imshow(fake_samples_scaled[looper,8,:,:])

In [ ]:
print(np.shape(dis_loss))
print(np.shape(dec_loss))
print(np.shape(enc_loss))

#dis_loss_plot = [row[0:4] for row in dis_loss]
#dec_loss_plot = ([row[0:4] for row in dec_loss]) 
#last_dec_row = [row[7] for row in dec_loss]
enc_loss_plot = enc_loss
dis_loss_plot = dis_loss
dec_loss_plot = dec_loss

width = 10
height = 5
plt.figure(figsize=(width,height))

plt.subplot(1,3,1)
plt.plot(dis_loss_plot)
#plt.legend(DCGAN_model.discriminator_train.metrics_names[0:3])
plt.legend(['classification_loss','real','decoded_real','fake'])
plt.ylim((0,10**1))
plt.title('Discriminator')

plt.subplot(1,3,2)
plt.plot(dec_loss_plot)
#plt.legend(DCGAN_model.decoder_train.metrics_names[0:4])
plt.legend(['classification_loss','recon_decoded_real','recon_fake','recon'])
plt.ylim((0,10**2))
plt.title('Decoder')

# plt.subplot(1,4,3)
# plt.plot(last_dec_row)
# plt.title('Decoder_weighted_bincross')

plt.subplot(1,3,3)
#plt.plot(enc_loss_plot[0,:] ,label='encoder')
plt.plot(enc_loss_plot)
#plt.ylim((-0,10**4))
plt.ylim((0,2))
plt.legend(DCGAN_model.encoder_train.metrics_names)
plt.title('Encoder')

In [ ]:
width = 10
height = 10
plt.figure(figsize=(width,height))
plotcounter = 1    
for looper in range(int(epochs/500)):
    plt.figure(figsize=(width,height))
    plt.subplot(3,3,plotcounter)
    plt.imshow(fake_samples_training[looper,0,:,:])
    plt.subplot(3,3,plotcounter+1)
    plt.imshow(fake_samples_training[looper,1,:,:])
    plt.subplot(3,3,plotcounter+2)
    plt.imshow(fake_samples_training[looper,2,:,:])
    plt.subplot(3,3,plotcounter+3)
    plt.imshow(fake_samples_training[looper,3,:,:])
    plt.subplot(3,3,plotcounter+4)
    plt.imshow(fake_samples_training[looper,4,:,:])
    plt.subplot(3,3,plotcounter+5)
    plt.imshow(fake_samples_training[looper,5,:,:])
    plt.subplot(3,3,plotcounter+6)
    plt.imshow(fake_samples_training[looper,6,:,:])
    plt.subplot(3,3,plotcounter+7)
    plt.imshow(fake_samples_training[looper,7,:,:])
    plt.subplot(3,3,plotcounter+8)
    plt.imshow(fake_samples_training[looper,8,:,:])

In [ ]:
#Check encoder
width = 10
height = 10
plt.figure(figsize=(width,height))

plotcounter = 1
imgs_org = x_train_small[range(1,batch_size)]
latents = DCGAN_model.encoder.predict(imgs_org)[2]
decoded = DCGAN_model.decoder.predict(latents)
print(np.amax(imgs_org))
print(np.amin(imgs_org))

print(np.amax(decoded))
print(np.amin(decoded))
plt.subplot(2,4,plotcounter)
plt.imshow((((imgs_org[0,:,:,:]*127.5)+127.5).astype(np.uint8)))
plt.subplot(2,4,plotcounter+1)
plt.imshow((((imgs_org[1,:,:,:]*127.5)+127.5).astype(np.uint8)))
plt.subplot(2,4,plotcounter+2)
plt.imshow((((imgs_org[2,:,:,:]*127.5)+127.5).astype(np.uint8)))
plt.subplot(2,4,plotcounter+3)
plt.imshow((((imgs_org[3,:,:,:]*127.5)+127.5).astype(np.uint8)))
plt.subplot(2,4,plotcounter+4)
plt.imshow((((decoded[0,:,:,:]*127.5)+127.5).astype(np.uint8)))
plt.subplot(2,4,plotcounter+5)
plt.imshow((((decoded[1,:,:,:]*127.5)+127.5).astype(np.uint8)))
plt.subplot(2,4,plotcounter+6)
plt.imshow((((decoded[2,:,:,:]*127.5)+127.5).astype(np.uint8)))
plt.subplot(2,4,plotcounter+7)
plt.imshow((((decoded[3,:,:,:]*127.5)+127.5).astype(np.uint8)))

# plt.subplot(2,4,plotcounter+4)
# plt.imshow(decoded[0,:,:,:])
# plt.subplot(2,4,plotcounter+5)
# plt.imshow(decoded[1,:,:,:])
# plt.subplot(2,4,plotcounter+6)
# plt.imshow(decoded[2,:,:,:])
# plt.subplot(2,4,plotcounter+7)
# plt.imshow(decoded[3,:,:,:])